In [20]:
import random
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from random import randint


In [26]:

id_1 = 7479 #change to first student id
id_2 = 7839 #change to second student id
id_3 = 0000 #change to third student id "leave 0000 if team of 2"
random_seed = id_1+id_2+id_3
random.seed(random_seed)
data_path="data.csv"#replace with data path
output_path="your_data.csv"#replace with output data path

all_data=pd.read_csv(data_path) 
all_columns = all_data.columns.tolist()

target_column = 'smoking'  

all_columns.remove(target_column)

selected_columns = random.sample(all_columns, 10)

print(selected_columns) #MUST BE PRINTED
selected_columns = np.append(selected_columns, target_column)
sample_df = all_data[selected_columns].copy()
sample_df.to_csv(output_path)   #From HERE YOU CAN SPLIT FOR TRAIN ,VALID AND TEST
df=sample_df


['fasting blood sugar', 'serum creatinine', 'Urine protein', 'age', 'weight(kg)', 'Cholesterol', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'triglyceride']


In [15]:
# GRID SEARCH 
# param_grid = [
#   {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
#   {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
#  ]
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
X=df.iloc[:,0:10]
Y=df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

tuned_parameters = [{'n_estimators': [50, 100, 200],
                    'learning_rate': [0.01, 0.1, 0.5, 1],
                    'algorithm': ['SAMME', 'SAMME.R'],
                    'base_estimator__max_depth': [1, 2, 3]}]

# cv number of times to run all possible answers
clf = GridSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=5,scoring='f1_macro')
clf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier()),
             param_grid=[{'algorithm': ['SAMME', 'SAMME.R'],
                          'base_estimator__max_depth': [1, 2, 3],
                          'learning_rate': [0.01, 0.1, 0.5, 1],
                          'n_estimators': [50, 100, 200]}],
             scoring='f1_macro')

In [27]:
clf.best_params_
clf.cv_results_
clf.cv_results_.keys()
y_true, y_pred = y_test, clf.predict(X_test)
print(accuracy_score(y_true, y_pred))
for param, score in zip(clf.cv_results_['params'], clf.cv_results_['mean_test_score']):
    print(param, score)

{'n_estimators': 200,
 'learning_rate': 0.5,
 'base_estimator__max_depth': 3,
 'algorithm': 'SAMME.R'}

In [22]:
# Randomized SEARCH 
import scipy
# n_iter-> 10 random combinations of the hyperparamaters
clf = RandomizedSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=5,n_iter=10,scoring='f1_macro')
clf.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier()),
                   param_distributions=[{'algorithm': ['SAMME', 'SAMME.R'],
                                         'base_estimator__max_depth': [1, 2, 3],
                                         'learning_rate': [0.01, 0.1, 0.5, 1],
                                         'n_estimators': [50, 100, 200]}],
                   scoring='f1_macro')

In [25]:
clf.best_params_
clf.cv_results_
clf.cv_results_.keys()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
for param, score in zip(clf.cv_results_['params'], clf.cv_results_['mean_test_score']):
    print(param, score)

              precision    recall  f1-score   support

           0       0.78      0.71      0.75     13359
           1       0.67      0.75      0.71     10529

    accuracy                           0.73     23888
   macro avg       0.73      0.73      0.73     23888
weighted avg       0.73      0.73      0.73     23888

{'n_estimators': 50, 'learning_rate': 0.5, 'base_estimator__max_depth': 2, 'algorithm': 'SAMME.R'} 0.7236740140046601
{'n_estimators': 200, 'learning_rate': 1, 'base_estimator__max_depth': 3, 'algorithm': 'SAMME.R'} 0.7241598571497487
{'n_estimators': 200, 'learning_rate': 0.5, 'base_estimator__max_depth': 2, 'algorithm': 'SAMME'} 0.7157972460544679
{'n_estimators': 100, 'learning_rate': 0.1, 'base_estimator__max_depth': 3, 'algorithm': 'SAMME.R'} 0.7262459435869651
{'n_estimators': 100, 'learning_rate': 0.01, 'base_estimator__max_depth': 3, 'algorithm': 'SAMME.R'} 0.7112505835210439
{'n_estimators': 200, 'learning_rate': 0.5, 'base_estimator__max_depth': 3, 'algor

In [40]:
# bayesian utilize the results from the previous step to decide which hyperparameter to evaluate next
# acquisition function search for the next set of hyperparameters the TPE algorithm utilizes this historical information to decide where to sample the next set of hyperparameters.
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset
X=df.iloc[:,0:10]
Y=df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# the objective function is the function that measures the performance of a machine learning model with a given set of hyperparameters
def objective(params):
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return {'loss': -accuracy, 'status': STATUS_OK}

# Define the search space for hyperparameters
space = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 200]),
    'max_depth': hp.choice('max_depth', [None, 1, 2, 3, 4, 5]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4])
}
# The Trials object keeps track of the hyperparameter configurations and their corresponding results across multiple iterations
# Perform hyperparameter optimization fmin best hyperparameter with the smallest loss 
# (TPE Tree of Parzen Estimators) search algorithm ->acquisition function
# max value-> max number of evaluations
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Get the best hyperparameters
best_params = hyperopt.space_eval(space, best)
print("Best Hyperparameters:", best_params)

# Evaluate the model with the best hyperparameters
best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with Best Hyperparameters:", accuracy)


100%|██████████| 50/50 [04:56<00:00,  5.93s/trial, best loss: -0.7272689216342934]
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Test Accuracy with Best Hyperparameters: 0.7283991962491627


In [30]:
# FINAL SYSTEM 
boosting_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),algorithm='SAMME.R',learning_rate=0.5,n_estimators=200,random_state=42)
boosting_model.fit(X_train, y_train)
y_pred_boosting = boosting_model.predict(X_test)
accuracy_boosting = accuracy_score(y_test, y_pred_boosting)
print(f"Boosting Accuracy: {accuracy_boosting}")


Boosting Accuracy: 0.728985264567984
